In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sps
import scipy.sparse.linalg

In [ ]:
from topo import IntervalTopo1D
from basis import LagrangeBasis1D, LobattoBasis1D
from mesh import Mesh1D
from assemble import simple_assembly, simple_build_rhs

In [ ]:
L = 2*np.pi
k = 5

def f(x):
    return np.sin(k*2*np.pi*x/L)

def f2(x):
    return -(k*2*np.pi/L)**2*f(x)

errs = {}
order_vals = [1,2,3,4,5]
for order in order_vals:
    errs[order] = []
    for n_elems in [16, 32]:

        nodes = np.linspace(0, L, n_elems+1)
        elem_to_node = np.zeros((n_elems, 2), dtype=np.int)
        elem_to_node[:,0] = np.arange(n_elems)
        elem_to_node[:,1] = np.arange(n_elems)+1

        topo  = IntervalTopo1D()
        basis = LobattoBasis1D(topo, order)
        mesh  = Mesh1D(topo, basis)
        mesh.build_mesh(nodes, elem_to_node, [0,-1])

        cub_points, cub_weights = topo.get_quadrature(order+1)

        Kloc = np.zeros((basis.n_dofs, basis.n_dofs),
                         dtype=np.double)
        cub_vals = basis.eval_ref(np.eye(basis.n_dofs),
                                  cub_points, d=1)

        for i in range(basis.n_dofs):
            for j in range(basis.n_dofs):
                Kloc[i,j] = np.sum(cub_vals[i]*cub_vals[j]*cub_weights)

        K = simple_assembly(mesh, Kloc)
        rhs = simple_build_rhs(topo, basis, mesh, f2)

        sol = sps.linalg.spsolve(K, rhs)

        x_vals = np.linspace(0, L, mesh.n_elems+1)
        x_vals = (x_vals[:-1]+x_vals[1:])/2.0
        s = basis.eval_ref(sol[mesh.elem_to_dof], np.array([0.0])).ravel()

        h = L/n_elems
        err = h*np.linalg.norm(s-f(x_vals))
        err = np.max(np.abs(s-f(x_vals)))
        errs[order].append(err)
    

In [ ]:
for order in order_vals:
    print order, -np.log2(errs[order][1]/errs[order][0])